In [26]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Python Spark SQL basic example").getOrCreate()

In [27]:
orders_rdd = spark.sparkContext.textFile("D:\GKD\SPARK_HDFS\data_etc\data.csv") 

In [28]:
orders_rdd.count()

10

In [29]:
orders_rdd.take(10) #hiển thị ra 10 dòng

['1,2013-07-25 00:00:00.0,11599,CLOSED',
 '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT',
 '3,2013-07-25 00:00:00.0,12111,COMPLETE',
 '4,2013-07-25 00:00:00.0,8827,CLOSED',
 '5,2013-07-25 00:00:00.0,11318,COMPLETE',
 '6,2013-07-25 00:00:00.0,7130,COMPLETE',
 '7,2013-07-25 00:00:00.0,4530,COMPLETE',
 '8,2013-07-25 00:00:00.0,2911,PROCESSING',
 '9,2013-07-25 00:00:00.0,5657,PENDING_PAYMENT',
 '10,2013-07-25 00:00:00.0,5648,PENDING_PAYMENT']

### số lượng order tương ứng với mỗi status COMPLETE,PENDING_PAYMENT,CLOSED...

In [30]:
map_rdd = orders_rdd.map(lambda x: (x.split(",")[3],1))

In [31]:
map_rdd.take(10)

[('CLOSED', 1),
 ('PENDING_PAYMENT', 1),
 ('COMPLETE', 1),
 ('CLOSED', 1),
 ('COMPLETE', 1),
 ('COMPLETE', 1),
 ('COMPLETE', 1),
 ('PROCESSING', 1),
 ('PENDING_PAYMENT', 1),
 ('PENDING_PAYMENT', 1)]

In [32]:
reduced_rdd = map_rdd.reduceByKey(lambda x,y: x+y)

In [33]:
reduced_rdd.take(10)

[('CLOSED', 2), ('PENDING_PAYMENT', 3), ('COMPLETE', 4), ('PROCESSING', 1)]

In [34]:
#hoặc dùng collect để hiển thị
reduced_rdd.collect()

[('CLOSED', 2), ('PENDING_PAYMENT', 3), ('COMPLETE', 4), ('PROCESSING', 1)]

In [35]:
sort_rdd = reduced_rdd.sortBy(lambda x: x[1]) #sort tại phần tử có index 1

In [36]:
sort_rdd.collect()

[('PROCESSING', 1), ('CLOSED', 2), ('PENDING_PAYMENT', 3), ('COMPLETE', 4)]

In [37]:
sort_rdd = reduced_rdd.sortBy(lambda x: x[1], False)
sort_rdd.collect()

[('COMPLETE', 4), ('PENDING_PAYMENT', 3), ('CLOSED', 2), ('PROCESSING', 1)]

### top khách hàng có orders nhiều nhất

In [38]:
cust_map_rdd = orders_rdd.map(lambda x: (x.split(",")[2],1))
cust_map_rdd.collect()

[('11599', 1),
 ('256', 1),
 ('12111', 1),
 ('8827', 1),
 ('11318', 1),
 ('7130', 1),
 ('4530', 1),
 ('2911', 1),
 ('5657', 1),
 ('5648', 1)]

In [39]:
reduced_cust_rdd = cust_map_rdd.reduceByKey(lambda x,y: x+y)
reduced_cust_rdd.collect()
# do data không đủ nên k trực quan

[('256', 1),
 ('12111', 1),
 ('11318', 1),
 ('7130', 1),
 ('2911', 1),
 ('5657', 1),
 ('11599', 1),
 ('8827', 1),
 ('4530', 1),
 ('5648', 1)]

In [40]:
sort_cust_rdd = reduced_cust_rdd.sortBy(lambda x:x[1], False) # giảm dần
sort_cust_rdd.collect()


[('256', 1),
 ('12111', 1),
 ('11318', 1),
 ('7130', 1),
 ('2911', 1),
 ('5657', 1),
 ('11599', 1),
 ('8827', 1),
 ('4530', 1),
 ('5648', 1)]

### tìmm tổng số lượng khách hàng


In [41]:
orders_rdd.count() #số dòng

10

In [42]:
cust_rdd = orders_rdd.map(lambda x: x.split(",")[2])
cust_rdd.count()


10

In [43]:
total_cust_rdd = cust_rdd.distinct()
total_cust_rdd.count()

10

### tìm khách hàng chỉ có closed nhiều nhất

In [44]:
filter_rdd = orders_rdd.filter(lambda x: x.split(",")[3]=='CLOSED')
filter_rdd.collect()

['1,2013-07-25 00:00:00.0,11599,CLOSED', '4,2013-07-25 00:00:00.0,8827,CLOSED']

In [45]:
cust_closed = filter_rdd.map(lambda x: (x.split(",")[2],1))
cust_closed.collect()

[('11599', 1), ('8827', 1)]

In [46]:
sort_cust_closed_rdd = cust_closed.sortBy(lambda x:x[1], False) # giảm dần
sort_cust_closed_rdd.take(1)

[('11599', 1)]

## reduce vs reducebykey


In [47]:
fruit = [("cam",3),("táo", 5),("chuối", 2), ("cam", 7), ("táo", 2), ("táo", 5)]

In [48]:
rdd = spark.sparkContext.parallelize(fruit) 
rdd.collect()

[('cam', 3), ('táo', 5), ('chuối', 2), ('cam', 7), ('táo', 2), ('táo', 5)]

In [49]:
total_fruit = rdd.values().reduce(lambda x,y : x+y)
print(total_fruit)

24


In [50]:
total_fruit_key = rdd.reduceByKey(lambda x,y : x+y)
total_fruit_key.collect()

[('cam', 10), ('chuối', 2), ('táo', 12)]